# Connect-Four AI - A Battle Between AI/ML Algorithms


Connect-Four is a simple board game: https://www.youtube.com/watch?v=ylZBRUJi3UQ

Caution: Just for fun!!
Haven't captured a few edge cases (i.e. wrong user input/invalid moves, index out of bound)
Will be fixed in the next release! :p :p

Game Rules:
Connect Four (also known as Four Up, Plot Four, Find Four, Four in a Row, Four in a Line, Drop Four, and Gravitrips in the Soviet Union) is a two-player connection board game, in which the players choose a color and then take turns dropping colored discs into a seven-column, six-row vertically suspended grid. The pieces fall straight down, occupying the lowest available space within the column. The objective of the game is to be the first to form a horizontal, vertical, or diagonal line of four of one's own discs. Connect Four is a solved game. The first player can always win by playing the right moves. Reference: Wikipedia- https://en.wikipedia.org/wiki/Connect_Four


In [1]:
import math
from random import randint
from sklearn.linear_model import LogisticRegression
import numpy as np
import tensorflow as tf
from tensorflow.python.keras import layers, models
import matplotlib.pyplot as plt
import sys, os

## Connect-Four Game Implementation

This game has two main characteristics- Move and Winner Check.
CheckWinner method will be called after each move.

Before executing move, validMove method will be called from the client side just to check if the move is valid.
If the move is valid, the move method will be called.
The sign will be planted in the board (based on given row and col)
Then the checkWinner method will be called.
Last thing we need to check is if the match is a draw!

Next thing is to check if the player is the winner after the last move.
Check-
 * if four consecutive rows contain same sign OR
 * if four consecutive columns contain same sign OR
 * if four consecutive diagonal or anti-diagonal boxes contain same sign

In [2]:
class ConnectFour:
    
    def __init__(self, size, players):
        self.size = size
        self.players = players
        self.board = [[' ' for j in range(size)] for i in range(size)]
        self.moves = []
        self.winner = None
        self.newData = []
    
    """
    Before executing move, validMove method will be called from the client side just to check if the move is valid.
    If the move is valid, the move method will be called.
    The sign will be planted in the board (based on given row and col)
    Then the checkWinner method will be called.
    Last thing we need to check is if the match is a draw!
    
    :param c: desire column where the player want to execute his next move
    :param player: it's a two player game, player id (1 or 2) will be passed through this parameter
    :return 1 if current player is winner, 2 if the other player is winner, 0 if it's a draw, -1 if move is invalid.
    """
    def move (self, c, player):
        r = self.lastAvailableRow (c)
        # print (r, c)
        
        if r == -1:
            return r
        
        sign = self.players[player-1]
        
        self.board[r][c] = sign
        
        self.moves.append([r, c])
        
        if self.checkWinner (r, c, player):
            # print("Player ", player, " wins!")
            return 1
        
        if self.checkDraw ():
            # print("Draw!")
            return 0
        
        # print("Next move please!")
        return 2
    
    def lastAvailableRow (self, c):
        r = 0
        while r < self.size and self.board[r][c] == ' ':
            r += 1
        
        r -= 1
        
        if r < 0:
            return -1
        
        return r
    
    """
    Check if the game is a draw. Check if the board is full.
    """
    def checkDraw (self):
        status = len(self.moves) == self.size * self.size
        if status:
            self.setWinner(0)
            
        return status
    
    """
    Check if the player is the winner after the last move.
    Check-
    * if all the rows contain same sign OR
    * if all the columns contain same sign OR
    * if all diagonal or anti-diagonal boxes contain same sign
    
    :param r: row where the player just executed their last move
    :param c: column where the player just executed their last move
    :param player: playerID (1 or 2) who has executed the last move
    :return true if the player is the winner, false otherwise
    """
    def checkWinner (self, r, c, player):
        status = self.checkRow (r, c, player) or self.checkCol (r, c, player) or self.checkDiagonal (r, c, player) or self.checkAntiDiagonal (r, c, player)
        
        if status:
            self.setWinner(player)
        return status
    
    """
    Check if all the rows contain same sign
    
    :param r row where the player just executed their last move
    :param player playerID (1 or 2) who has executed the last move
    :return true if all the rows contain same sign, false otherwise
    """
    def checkRow(self, r, c, player):
        count = 1
        currCol = c-1
        
        while currCol >= 0 and self.board[r][currCol] == self.players[player-1]:
            currCol -= 1
            count += 1
        
        currCol = c+1
        while currCol < self.size and self.board[r][currCol] == self.players[player-1]:
            currCol += 1
            count += 1
        
        return True if count >= 4 else False
    
    """
    Check if all the columns contain same sign
    
    :param c column where the player just executed their last move
    :param player playerID (1 or 2) who has executed the last move
    :return true if all the columns contain same sign, false otherwise
    """
    def checkCol(self, r, c, player):
        if self.size - r < 4:
            return False
        
        count = 1
        currRow = r+1
        while currRow < self.size and self.board[currRow][c] == self.players[player-1]:
            currRow += 1
            count += 1
        
        return True if count >= 4 else False
    
    """
    Check if all diagonal or anti-diagonal boxes contain same sign
    
    :param player playerID (1 or 2) who has executed the last move
    :return true if all diagonal or anti-diagonal boxes contain same sign, false otherwise
    """
    def checkDiagonal (self, r, c, player):
        count = 1
        
        currR = r + 1
        currC = c + 1
        
        while currR < self.size and currC < self.size :
            if self.board[currR][currC] == self.players[player-1]:
                count += 1
            else:
                break
            currR += 1
            currC += 1
            
        currR = r - 1
        currC = c - 1
        
        while currR >= 0 and currC >= 0:
            if self.board[currR][currC] == self.players[player-1]:
                count += 1
            else:
                break
                
            currR -= 1
            currC -= 1
        
        return True if count >= 4 else False
    
    """
    Check anti-diagonals
    """
    def checkAntiDiagonal (self, r, c, player):
        count = 1
        
        currR = r + 1
        currC = c - 1
        
        while currR < self.size and currC >= 0 :
            if self.board[currR][currC] == self.players[player-1]:
                count += 1
            else:
                break
            currR += 1
            currC -= 1
            
        currR = r - 1
        currC = c + 1
        
        while currR >= 0 and currC < self.size:
            if self.board[currR][currC] == self.players[player-1]:
                count += 1
            else:
                break
        
            currR -= 1
            currC += 1
        
        return True if count >= 4 else False
    
    """
    Set winner once the match is done. 
    winner = 1 for player 1, 2 for player 2, 0 for draw
    """
    def setWinner (self, winner):
        self.winner = winner
    
    """
    Return the winner.
    winner = 1 for player 1, 2 for player 2, 0 for draw
    """
    def getWinner (self):
        return self.winner
    
    """
    Check if the move is valid.
    In other word, check if the given position (row, col) in the board is empty
    
    :param r desire row where the player want to execute his next move
    :param c desire column where the player want to execute his next move
    :return true if the move is valid, false otherwise.
    """
    def isValidMove (self, c):
        return False if self.lastAvailableRow(c) == -1 else True
    
    """
    Undo last move!
    """
    def undo (self):
        if len(self.moves) > 0:
            lastMove = self.moves.pop(-1)
            self.board[lastMove[0]][lastMove[1]] = ' '
            self.setWinner(None)
            # self.displayBoard()
    
    """
    Display the Connect-Four Board (array)
    """
    def displayBoard(self):
        
        seperator = []
        for i in range (self.size - 1):
            seperator.append('-')
            seperator.append('+')
        seperator.append('-')
        
        print()
        for row, val in enumerate(self.board):
            print(*val, sep = " | ")
            if not row == self.size - 1:
                print(*seperator)
        print()
        
    """
    Add new row to temporary dataset (newData) after every move. 
    This is using to generate new data so that we can train our machine learning model with new data after each game.
    
    This function creates two rows for each move considering both of the players as winner 
    
    """
    def generateNewRow(self):
        newRow = []
        for row in range (self.size):
            for col in range (self.size):
                val = 0
                if (self.board[row][col] == self.players[0]):
                    val = 1
                elif (self.board[row][col] == self.players[1]):
                    val = -1
                
                newRow.append(val)
        
        newInvertRow = [v if v == 0 else -1 if v == 1 else 1 for v in newRow]
        
        self.newData.append (newRow)
        self.newData.append (newInvertRow)
        
        # print (self.newData)
        
    """
    getNewData will be called at the end of each match. 
    This function labels all the data and return a new set of dataset so that we can train our ML model with this new set of data
    
    """
    def getNewData (self, winner):
        if (winner == 1): 
            newTrainY = [1 if i % 2 == 0 else 2 for i in range(len(self.newData))]
        elif (winner == 2): 
            newTrainY = [2 if i % 2 == 0 else 1 for i in range(len(self.newData))]
        else: 
            newTrainY = [0 for i in range(len(self.newData))]
        
        newTrainX = self.newData
        self.newData = []
        
        print("Size of newTrainX and newTrainY: ", len(newTrainX), len(newTrainY))
        # print(newTrainX)
        # print(newTrainY)
        
        return newTrainX, newTrainY
    
    """
    Reset the variables to make the board ready the next match!
    """
    def clear (self, size, players):
        self.size = size
        self.players = players
        self.board = [[' ' for j in range(size)] for i in range(size)]
        self.moves = []
        self.winner = None

Below is the main function for the normal two player Connect-Four game. Nothing fancy. Just wanted to check if my implementation works properly. And guess what? It works!!

In [3]:
def main(): 
    print("choose a board size: ")
    size = input()
    
    print("Pick your lucky color: R or B")
    symbol = input()
    
    players = []
    if (symbol == 'R'):
        players.append('R')
        players.append('B')
    else:
        players.append('B')
        players.append('R')
    
    game = ConnectFour(int(size), players)
    game.displayBoard()
    
    player = 1
    print("Let's begin..")
    
    while (True):
        print("Player ", player, "'s turn!")
        print("Choose a column betwwen 1 to ", game.size)
        inputdata = input()
        
        if (inputdata == "exit"):
            break
            
        if (inputdata == "z"): #undo
            game.undo()
            player = 2 if (player == 1) else 1
            continue
            
        c = int(inputdata) - 1
        result = game.move(c, player)
        game.displayBoard()
        
        if (result == -1):
            print("********************************************************************************")
            print("Come on! Invalid move, dude! Okay, I am giving you another chance. Concentrate!!")
            print("********************************************************************************")
            continue
        
        if result == 0 or result == 1:
            print("===============")
            print("One more? (y/n)")
            print("===============")
            isNewGame = input()
            if (isNewGame == 'y'):
                game = ConnectFour(int(size), players)
                continue
            else:
                break
        
        player = 2 if (player == 1) else 1
    
  
if __name__=="__main__": 
    main() 

choose a board size: 
9
Pick your lucky color: R or B
B

  |   |   |   |   |   |   |   |  
- + - + - + - + - + - + - + - + -
  |   |   |   |   |   |   |   |  
- + - + - + - + - + - + - + - + -
  |   |   |   |   |   |   |   |  
- + - + - + - + - + - + - + - + -
  |   |   |   |   |   |   |   |  
- + - + - + - + - + - + - + - + -
  |   |   |   |   |   |   |   |  
- + - + - + - + - + - + - + - + -
  |   |   |   |   |   |   |   |  
- + - + - + - + - + - + - + - + -
  |   |   |   |   |   |   |   |  
- + - + - + - + - + - + - + - + -
  |   |   |   |   |   |   |   |  
- + - + - + - + - + - + - + - + -
  |   |   |   |   |   |   |   |  

Let's begin..
Player  1 's turn!
Choose a column betwwen 1 to  9
3

  |   |   |   |   |   |   |   |  
- + - + - + - + - + - + - + - + -
  |   |   |   |   |   |   |   |  
- + - + - + - + - + - + - + - + -
  |   |   |   |   |   |   |   |  
- + - + - + - + - + - + - + - + -
  |   |   |   |   |   |   |   |  
- + - + - + - + - + - + - + - + -
  |   |   |   |   |   | 

## Connect-Four AI

Extending the basic implementation of Connect-Four and adding necessary features to make it compitable with AI system. 

In [3]:
class ConnectFourAI (ConnectFour):
    
    def __init__(self, size, players):
        super().__init__(size, players)
        
    def allPossibleNextMoves (self):
        possibleMoves = []
        
        for col in range(self.size):
            if (self.isValidMove(col)):
                possibleMoves.append(col)
        
        return possibleMoves

## AI/ML Algorithms

In [4]:
class AIAlgorithms:
    def __init__(self, game):
        self.game = game
        self.dataset = []

## MiniMaxAlgorithm

In [5]:
class MiniMaxAlgorithm (AIAlgorithms):
    
    def __init__(self, game):
        super().__init__(game)
    """
    Thanks to wiki,The Coding Train and levelup: 
    https://en.wikipedia.org/wiki/Minimax
    https://levelup.gitconnected.com/mastering-tic-tac-toe-with-minimax-algorithm-3394d65fa88f
    https://www.youtube.com/watch?v=trKjYdBASyQ&t=1196s
    """
    
    """
    Iterate through all the possible moves and call minimax each time to find the best possible move
    """
    def findBestMiniMaxMove (self, player):
        bestScore = -math.inf
        bestMove = None
        counter = [0]
        
        for possibleMove in self.game.allPossibleNextMoves():
            self.game.move(possibleMove, player)
            score = self.minimax (False, player, 0, counter)
            self.game.undo()

            if (score > bestScore):
                bestScore = score
                bestMove = possibleMove
        
        print ("I have compared ", counter[0], " combinations and executed the best move out of it. I can't lose, dude!")
        return bestMove
    
    """
    Return Max Score and Min Score respectively for Maximizing and Minimizing player.
    """
    def minimax (self, isMax, player, depth, counter):
        
        if depth == 5:
            return 0
        
        counter[0] = counter[0] + 1
        
        winner = self.game.getWinner()
        if (not (winner == None)):
            if (winner == 0):
                return 0
            elif (winner == player):
                return 10 - depth
            else:
                return depth - 10
        
        maxScore = -math.inf
        minScore = math.inf
        
        for possibleMove in self.game.allPossibleNextMoves():
            currPlayer = player if isMax else 2 if (player == 1) else 1
            
            self.game.move(possibleMove, currPlayer)
            score = self.minimax (not isMax, player, depth + 1, counter)
            self.game.undo()
            
            if (score > maxScore):
                maxScore = score
            if (score < minScore):
                minScore = score
        
        return maxScore if isMax else minScore
    

## LogisticRegressionAlgorithm

In [12]:
class LogisticRegressionAlgorithm (AIAlgorithms):
    
    def __init__(self, game):
        super().__init__(game)
        self.LRModel = None
        self.trainX = []
        self.trainY = []
    
    """
    Iterate through all the possible moves, generate new test data and call logistic regression algorithm to find the best possible move based on the probability of winning.
    """
    def findBestLogisticMove (self, player):
        testX = []
        positions = []
        
        for possibleMove in self.game.allPossibleNextMoves():
            self.game.move(possibleMove, player)
            positions.append (possibleMove)
            testX.append (self.generateTestX())
            self.game.undo()
        
        index = 1 if player == 1 else 2
        
        predictions = np.around(self.logisticRegressionTesting (testX), decimals=2)
        
        maxProb = np.amax(predictions[:, index])
        moveIndex = np.where(predictions[:, index] == maxProb)[0][0]
        """
        print (predictions)
        print (self.LRModel.classes_)
        print (index)
        print (maxProb)
        print (moveIndex)
        """
        return positions[moveIndex]
    
    """
    Generating testing data based on all possible moves.
    """
    def generateTestX (self):
        newRow = []
        for row in range (self.game.size):
            for col in range (self.game.size):
                val = 0
                if (self.game.board[row][col] == self.game.players[0]):
                    val = 1
                elif (self.game.board[row][col] == self.game.players[1]):
                    val = -1
                
                newRow.append(val)
        return newRow
    
    """
    Train your logistic Regression model with the new and old dataset
    """
    def logisticRegressionTraining (self):
        dataset = np.concatenate((np.asarray(self.trainX), np.asarray([self.trainY]).T), axis=1)
        np.random.shuffle(dataset)
        
        X = dataset[:, :-1]
        y = dataset[:, -1]
        self.LRModel = LogisticRegression(random_state=0).fit(X, y)
        
    """
    Test your logistic Regression model with all possible moves and find the best move based on the probability of winning.
    """
    def logisticRegressionTesting (self, testX):
        return self.LRModel.predict_proba(np.asarray(testX))
    
    def learningTime (self):
        print ("Logictic Regression: Hey! It's learning time. Running towards being a human!")
        self.logisticRegressionTraining()
        print ("I have learnt lots of new tricks!")
    
    def addNewData (self, newTrainX, newTrainY):
        
        for i in range (len(newTrainX)):
            self.trainX.append(newTrainX[i])
            self.trainY.append(newTrainY[i])
        
        print("Training size: ", len(self.trainX))
        # self.learningTime()

## ConvolutionalNeuralNetworkAlgorithm

Well, I understand that Convolutional Neural Network is not worthy for a small board like Connect-Four. But I wanted to explore it anyway for few reasons. First, I wanted to check how CNN perform with small grids. Second, I didn't want to miss the opportunity to experiment with different network design. And Finally, this would give me a framework that I can use other board game later. Enough excuses, huh?

Thanks to Tensorflow. Tensorflow documentation is always well documented: https://www.tensorflow.org/tutorials/images/cnn

In [21]:
class ConvolutionalNeuralNetworkAlgorithm (AIAlgorithms):
    
    def __init__(self, game):
        super().__init__(game)
        self.model = None
        self.trainX = []
        self.trainY = []
        
    """
    Iterate through all the possible moves, generate new test data and call CNN algorithm to find the best possible move based on the probability of winning.
    """
    def findBestCNNMove (self, player):
        testX = []
        positions = []
        accuracy = []
        
        desireClass = 1 if player == 1 else 2
        
        for possibleMove in self.game.allPossibleNextMoves():
            self.game.move(possibleMove, player)
            positions.append (possibleMove)
            test_loss, test_acc = self.convolutionalNeuralNetworkTesting ([np.asarray(self.generateTestX()).reshape(self.game.size, self.game.size, 1)], [desireClass])
            accuracy.append (test_acc)
            self.game.undo()
        
        # print(accuracy)
        maxProb = np.amax(accuracy)
        # print(maxProb)
        moveIndex = np.where(accuracy == maxProb)[0][0]
        # print(moveIndex)
        """
        print (predictions)
        print (self.LRModel.classes_)
        print (index)
        print (maxProb)
        print (moveIndex)
        """
        return positions[moveIndex]
    
    """
    Generating testing data based on all possible moves.
    """
    def generateTestX (self):
        newRow = []
        for row in range (self.game.size):
            for col in range (self.game.size):
                val = 0
                if (self.game.board[row][col] == self.game.players[0]):
                    val = 1
                elif (self.game.board[row][col] == self.game.players[1]):
                    val = -1
                
                newRow.append(val)
        return newRow
    
    def convolutionalNeuralNetworkTraining (self):
        self.model = models.Sequential()
        self.model.add(layers.Conv2D(32, (2, 2), activation='relu', padding="same", input_shape=(self.game.size, self.game.size, 1)))
        self.model.add(layers.MaxPooling2D((2, 2), padding="same")) # dim_ordering="th"
        self.model.add(layers.Conv2D(64, (2, 2), activation='relu', padding="same"))
        self.model.add(layers.MaxPooling2D((2, 2), padding="same"))
        self.model.add(layers.Conv2D(32, (2, 2), activation='relu', padding="same"))
        self.model.add(layers.MaxPooling2D((2, 2), padding="same"))
        self.model.add(layers.Conv2D(16, (2, 2), activation='relu', padding="same"))
        
        self.model.add(layers.Flatten())
        self.model.add(layers.Dense(32, activation='relu'))
        self.model.add(layers.Dense(3, activation = "softmax"))
        
        self.model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
        
        print(self.model.summary())
        
        # img_count = len(self.trainX) 
        # print(self.trainX)
        history = self.model.fit(np.asarray(self.trainX), np.asarray(self.trainY), epochs=30, shuffle=True)
    
    def convolutionalNeuralNetworkTesting (self, test_images,  test_labels):
        # print(test_images)
        return self.model.evaluate(np.asarray(test_images),  np.asarray(test_labels), verbose=0)
        
    def learningTimeCNN (self):
        print ("ConvNet: Well, it's time to learn! My only mission is to replace human! Kidding.. KIDDING.. Trust me!")
        self.convolutionalNeuralNetworkTraining()
        print ("I have learnt lots of new tricks!")
        
    def addNewDataCNN (self, newTrainX, newTrainY):
        
        for i in range (len(newTrainX)):
            self.trainX.append(np.asarray(newTrainX[i]).reshape(self.game.size, self.game.size, 1)) #
            self.trainY.append(newTrainY[i])
        
        print("Training size: ", len(self.trainX))
        # self.learningTime()

## Random Moves Generator 

This function was used for generating data.

In [13]:
class RandomPlacementAlgorithm (AIAlgorithms):
    
    def __init__(self, game):
        super().__init__(game)
        self.LRModel = None
        
    def randomPlacement (self):
        possibleMoves = self.game.allPossibleNextMoves()
        index = randint(0, len(possibleMoves) - 1)
        return possibleMoves[index]
        

## Battle 1: Human Vs Minimax Algorithm!

In [11]:
def main(): 
    print("choose a board size: ")
    size = input()
    
    print("Pick your lucky color: R or B")
    symbol = input()
    
    players = []
    if (symbol == 'R'):
        players.append('R')
        players.append('B')
    else:
        players.append('B')
        players.append('R')
    
    game = ConnectFourAI(int(size), players)
    game.displayBoard()
    
    algorithm = MiniMaxAlgorithm(game)
    
    player = randint(1, 2)
    print("Let's begin..")
    
    while (True):
        
        result = None
        
        if player == 1:
            print ("It's human's turn!")
        if player == 2:
            print ("It's MiniMax's turn!")
        
        if player == 1:
            print("Choose a number betwwen 1 to ", game.size)
            inputdata = input()
            if (inputdata == "exit"):
                break
            
            if (inputdata == "z"): #undo
                game.undo()
                player = 2 if (player == 1) else 1
                continue

            c = int(inputdata) - 1
            result = game.move(c, player)
            
            if (result == -1):
                print("********************************************************************************")
                print("Come on! Invalid move, dude! Okay, I am giving you another chance. Concentrate!!")
                print("********************************************************************************")
                continue
            
        elif player == 2:
            bestMove = algorithm.findBestMiniMaxMove (player)
            result = game.move(bestMove, player)

        game.displayBoard()
        
#         if result == 0 or result == 1:
        if game.getWinner() == 0:
            print("Match Draw!")
            break
        elif game.getWinner() == 1:
            print("Human wins!")
            break
        elif game.getWinner() == 2:
            print("MiniMax wins!")
            break
                
        if result == 0 or result == 1:
            print("===============")
            print("One more? (y/n)")
            print("===============")
            isNewGame = input()
            if (isNewGame == 'y'):
                game = ConnectFourAI(int(size), players)
                game.displayBoard()
                algorithm = AIAlgorithms(game)
                continue
            else:
                break
        
        player = 2 if (player == 1) else 1
    
  
if __name__=="__main__": 
    main() 

choose a board size: 
5
Pick your lucky color: R or B
R

  |   |   |   |  
- + - + - + - + -
  |   |   |   |  
- + - + - + - + -
  |   |   |   |  
- + - + - + - + -
  |   |   |   |  
- + - + - + - + -
  |   |   |   |  

Let's begin..
It's MiniMax's turn!
I have compared  97505  combinations and executed the best move out of it. I can't lose, dude!

  |   |   |   |  
- + - + - + - + -
  |   |   |   |  
- + - + - + - + -
  |   |   |   |  
- + - + - + - + -
  |   |   |   |  
- + - + - + - + -
B |   |   |   |  

It's human's turn!
Choose a number betwwen 1 to  5
2

  |   |   |   |  
- + - + - + - + -
  |   |   |   |  
- + - + - + - + -
  |   |   |   |  
- + - + - + - + -
  |   |   |   |  
- + - + - + - + -
B | R |   |   |  

It's MiniMax's turn!
I have compared  95314  combinations and executed the best move out of it. I can't lose, dude!

  |   |   |   |  
- + - + - + - + -
  |   |   |   |  
- + - + - + - + -
  |   |   |   |  
- + - + - + - + -
B |   |   |   |  
- + - + - + - + -
B | R | 

## Battle 2: Minimax Vs Minimax!

In [18]:
def main():    
    player = randint(1, 2)
    print("Let's begin..")
    
    game = ConnectFourAI(6, ['X', 'O'])
    game.displayBoard()
    algorithm = MiniMaxAlgorithm(game)
    
    while (True):
        
        result = None
        
        if player == 1:
            print ("It's MiniMax1's turn!")
        if player == 2:
            print ("It's MiniMax2's turn!")
        
        bestMove = algorithm.findBestMiniMaxMove (player)
        result = game.move(bestMove, player)
        game.displayBoard()
        
        if game.getWinner() == 0:
            print("Match Draw!")
            break
        elif game.getWinner() == 1:
            print("Minimax1 wins!")
            break
        elif game.getWinner() == 2:
            print("MiniMax2 wins!")
            break
        
        player = 2 if (player == 1) else 1

if __name__=="__main__": 
    main() 

Let's begin..

  |   |   |   |   |  
- + - + - + - + - + -
  |   |   |   |   |  
- + - + - + - + - + -
  |   |   |   |   |  
- + - + - + - + - + -
  |   |   |   |   |  
- + - + - + - + - + -
  |   |   |   |   |  
- + - + - + - + - + -
  |   |   |   |   |  

It's MiniMax1's turn!
I have compared  9330  combinations and executed the best move out of it. I can't lose, dude!

  |   |   |   |   |  
- + - + - + - + - + -
  |   |   |   |   |  
- + - + - + - + - + -
  |   |   |   |   |  
- + - + - + - + - + -
  |   |   |   |   |  
- + - + - + - + - + -
  |   |   |   |   |  
- + - + - + - + - + -
X |   |   |   |   |  

It's MiniMax2's turn!
I have compared  9330  combinations and executed the best move out of it. I can't lose, dude!

  |   |   |   |   |  
- + - + - + - + - + -
  |   |   |   |   |  
- + - + - + - + - + -
  |   |   |   |   |  
- + - + - + - + - + -
  |   |   |   |   |  
- + - + - + - + - + -
O |   |   |   |   |  
- + - + - + - + - + -
X |   |   |   |   |  

It's MiniMax1's turn!


- + - + - + - + - + -
X | X | O | O | X | X
- + - + - + - + - + -
O | O | X | X | O | O
- + - + - + - + - + -
X | X | O | O | X | X
- + - + - + - + - + -
O | O | X | O | O | O
- + - + - + - + - + -
X | X | X | O | X | X

It's MiniMax2's turn!
I have compared  1  combinations and executed the best move out of it. I can't lose, dude!

O | O | X | X | O | O
- + - + - + - + - + -
X | X | O | O | X | X
- + - + - + - + - + -
O | O | X | X | O | O
- + - + - + - + - + -
X | X | O | O | X | X
- + - + - + - + - + -
O | O | X | O | O | O
- + - + - + - + - + -
X | X | X | O | X | X

Match Draw!


## Battle 3: Minimax Vs Logistic Regression!

Below is the output of Minimax Vs Logistic Regression!. Minimax is always unbeatable. Logistic Regression is unbeatable as well if it is trained with good dataset. Logistic Regression is way faster than Minimax (obviously minimax would perform faster with alpha-beta prouning.) Logistic Regression can lose first few matches if the model is not trained well, but we are training the model at the end of each match, so logistic regression will definitely make a come back! Fingers Crossed!! 

Data has been gereated by placing some random moves against minimax algorithm.

In [20]:
def main():    
    player = randint(1, 2)
    print("Let's begin..")
    
    game = ConnectFourAI(6, ['X', 'O'])
#     game.displayBoard()
    
    miniMaxAlgorithm = MiniMaxAlgorithm(game)
    logisticRegressionAlgorithm = LogisticRegressionAlgorithm(game)
    randomPlacementAlgorithm = RandomPlacementAlgorithm(game)
    
    print("======================== Generating data =================================")
    
    # 
    generateData (player, game, miniMaxAlgorithm, logisticRegressionAlgorithm, randomPlacementAlgorithm)
    # 
    
    print("======================== Data has been generated =================================")
    
    logisticRegressionAlgorithm.learningTime()
    
    print("======================== IT'S BATTLE TIME =================================")
    
    mmCount = 0
    lrCount = 0
    drawCount = 0
    for i in range (5):
        print ("Battle number: ", i + 1)
        print ("==============================================================")
        print ("Minimax: ", mmCount, "Logistic Regression: ", lrCount, "Draw: ", drawCount)
        print ("==============================================================")
        game.clear(6, ['X', 'O'])
        
        while (True):
        
            result = None

            if player == 1:
                print ("It's MiniMax's turn!")
                bestMove = miniMaxAlgorithm.findBestMiniMaxMove (player)
                result = game.move(bestMove, player)

            if player == 2:
                print ("It's Logistic Regression's turn!")
                bestMove = logisticRegressionAlgorithm.findBestLogisticMove (player)
                result = game.move(bestMove, player)

            game.generateNewRow()
            game.displayBoard()

            if not result == 2:
                print ("It's over!")
                newTrainX, newTrainY = game.getNewData(game.getWinner())
                
                logisticRegressionAlgorithm.addNewData(newTrainX, newTrainY)
                logisticRegressionAlgorithm.learningTime()


            if game.getWinner() == 0:
                print("Match Draw!")
                drawCount += 1
                break
            elif game.getWinner() == 1:
                print("Minimax wins!")
                mmCount += 1
                break
            elif game.getWinner() == 2:
                print("Logistic Regression wins!")
                lrCount += 1
                break

            player = 2 if (player == 1) else 1
    print ("==============================================================")
    print ("Minimax: ", mmCount, "Logistic Regression: ", lrCount, "Draw: ", drawCount)
    print ("==============================================================")
    
def generateData (player, game, miniMaxAlgorithm, logisticRegressionAlgorithm, randomPlacementAlgorithm):
    # blockPrint()
    
    # Generating winning and losing data
    for i in range (10):
        game.clear(6, ['X', 'O'])
        
        while (True):
        
            result = None

            if player == 1:
                # print ("It's MiniMax's turn!")
                bestMove = miniMaxAlgorithm.findBestMiniMaxMove (player)
                result = game.move(bestMove, player)

            if player == 2:
                # print ("It's Random Placement's turn!")
                bestMove = randomPlacementAlgorithm.randomPlacement ()
                result = game.move(bestMove, player)

            game.generateNewRow()
            # game.displayBoard()

            if not result == 2:
                # print ("It's over!")
                newTrainX, newTrainY = game.getNewData(game.getWinner())
                
                logisticRegressionAlgorithm.addNewData(newTrainX, newTrainY)


            if game.getWinner() == 0:
                # print("Match Draw!")
                break
            elif game.getWinner() == 1:
                # print("Minimax wins!")
                break
            elif game.getWinner() == 2:
                # print("Random Placement wins!")
                break

            player = 2 if (player == 1) else 1
    
    # Generating data for draw matches
    for i in range (5):
        game.clear(6, ['X', 'O'])
        
        while (True):
        
            result = None
            
#             if player == 1:
#                 print ("It's MiniMax1's turn!")
#             if player == 2:
#                 print ("It's MiniMax2's turn!")

            bestMove = miniMaxAlgorithm.findBestMiniMaxMove (player)
            result = game.move(bestMove, player)

            game.generateNewRow()
            # game.displayBoard()

            if not result == 2:
                # print ("It's over!")
                newTrainX, newTrainY = game.getNewData(game.getWinner())
                
                logisticRegressionAlgorithm.addNewData(newTrainX, newTrainY)


            if game.getWinner() == 0:
                # print("Match Draw!")
                break
            elif game.getWinner() == 1:
                # print("Minimax1 wins!")
                break
            elif game.getWinner() == 2:
                # print("Minimax2 wins!")
                break

            player = 2 if (player == 1) else 1
    
    # unblockPrint()
    
# Disable print
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

# Enable print
def unblockPrint():
    sys.stdout = sys.__stdout__

if __name__=="__main__": 
    main() 

Let's begin..
======================== Generating data =================================
I have compared  9330  combinations and executed the best move out of it. I can't lose, dude!
I have compared  9090  combinations and executed the best move out of it. I can't lose, dude!
I have compared  7145  combinations and executed the best move out of it. I can't lose, dude!
I have compared  8974  combinations and executed the best move out of it. I can't lose, dude!
I have compared  6383  combinations and executed the best move out of it. I can't lose, dude!
Size of newTrainX and newTrainY:  20 20
Training size:  20
I have compared  9330  combinations and executed the best move out of it. I can't lose, dude!
I have compared  9330  combinations and executed the best move out of it. I can't lose, dude!
I have compared  8891  combinations and executed the best move out of it. I can't lose, dude!
I have compared  6480  combinations and executed the best move out of it. I can't lose, dude!
Size o

I have compared  2857  combinations and executed the best move out of it. I can't lose, dude!
I have compared  1244  combinations and executed the best move out of it. I can't lose, dude!
I have compared  1004  combinations and executed the best move out of it. I can't lose, dude!
I have compared  1316  combinations and executed the best move out of it. I can't lose, dude!
I have compared  1295  combinations and executed the best move out of it. I can't lose, dude!
I have compared  1295  combinations and executed the best move out of it. I can't lose, dude!
I have compared  1126  combinations and executed the best move out of it. I can't lose, dude!
I have compared  876  combinations and executed the best move out of it. I can't lose, dude!
I have compared  315  combinations and executed the best move out of it. I can't lose, dude!
I have compared  248  combinations and executed the best move out of it. I can't lose, dude!
I have compared  323  combinations and executed the best move o

I have compared  9330  combinations and executed the best move out of it. I can't lose, dude!
I have compared  9330  combinations and executed the best move out of it. I can't lose, dude!
I have compared  9329  combinations and executed the best move out of it. I can't lose, dude!
I have compared  9303  combinations and executed the best move out of it. I can't lose, dude!
I have compared  9032  combinations and executed the best move out of it. I can't lose, dude!
I have compared  7616  combinations and executed the best move out of it. I can't lose, dude!
I have compared  3905  combinations and executed the best move out of it. I can't lose, dude!
I have compared  3785  combinations and executed the best move out of it. I can't lose, dude!
I have compared  3664  combinations and executed the best move out of it. I can't lose, dude!
I have compared  3763  combinations and executed the best move out of it. I can't lose, dude!
I have compared  3468  combinations and executed the best mo

I have compared  9303  combinations and executed the best move out of it. I can't lose, dude!

  |   |   |   |   |  
- + - + - + - + - + -
  |   |   |   |   |  
- + - + - + - + - + -
X |   |   |   |   |  
- + - + - + - + - + -
O |   |   |   |   |  
- + - + - + - + - + -
X |   |   |   |   |  
- + - + - + - + - + -
O |   |   |   |   |  

It's Logistic Regression's turn!

  |   |   |   |   |  
- + - + - + - + - + -
  |   |   |   |   |  
- + - + - + - + - + -
X |   |   |   |   |  
- + - + - + - + - + -
O |   |   |   |   |  
- + - + - + - + - + -
X |   |   |   |   |  
- + - + - + - + - + -
O |   |   |   |   | O

It's MiniMax's turn!
I have compared  9032  combinations and executed the best move out of it. I can't lose, dude!

  |   |   |   |   |  
- + - + - + - + - + -
X |   |   |   |   |  
- + - + - + - + - + -
X |   |   |   |   |  
- + - + - + - + - + -
O |   |   |   |   |  
- + - + - + - + - + -
X |   |   |   |   |  
- + - + - + - + - + -
O |   |   |   |   | O

It's Logistic Regression's

I have compared  7616  combinations and executed the best move out of it. I can't lose, dude!

X |   |   |   |   |  
- + - + - + - + - + -
X |   |   |   |   |  
- + - + - + - + - + -
O |   |   |   |   |  
- + - + - + - + - + -
X |   |   |   |   |  
- + - + - + - + - + -
O |   |   |   |   |  
- + - + - + - + - + -
X | O |   |   |   |  

It's Logistic Regression's turn!

X |   |   |   |   |  
- + - + - + - + - + -
X |   |   |   |   |  
- + - + - + - + - + -
O |   |   |   |   |  
- + - + - + - + - + -
X |   |   |   |   |  
- + - + - + - + - + -
O | O |   |   |   |  
- + - + - + - + - + -
X | O |   |   |   |  

It's MiniMax's turn!
I have compared  3814  combinations and executed the best move out of it. I can't lose, dude!

X |   |   |   |   |  
- + - + - + - + - + -
X |   |   |   |   |  
- + - + - + - + - + -
O |   |   |   |   |  
- + - + - + - + - + -
X | X |   |   |   |  
- + - + - + - + - + -
O | O |   |   |   |  
- + - + - + - + - + -
X | O |   |   |   |  

It's Logistic Regression's

I have compared  9303  combinations and executed the best move out of it. I can't lose, dude!

  |   |   |   |   |  
- + - + - + - + - + -
  |   |   |   |   |  
- + - + - + - + - + -
X |   |   |   |   |  
- + - + - + - + - + -
O |   |   |   |   |  
- + - + - + - + - + -
X |   |   |   |   |  
- + - + - + - + - + -
O |   |   |   |   |  

It's Logistic Regression's turn!

  |   |   |   |   |  
- + - + - + - + - + -
  |   |   |   |   |  
- + - + - + - + - + -
X |   |   |   |   |  
- + - + - + - + - + -
O |   |   |   |   |  
- + - + - + - + - + -
X |   |   |   |   |  
- + - + - + - + - + -
O | O |   |   |   |  

It's MiniMax's turn!
I have compared  8794  combinations and executed the best move out of it. I can't lose, dude!

  |   |   |   |   |  
- + - + - + - + - + -
X |   |   |   |   |  
- + - + - + - + - + -
X |   |   |   |   |  
- + - + - + - + - + -
O |   |   |   |   |  
- + - + - + - + - + -
X |   |   |   |   |  
- + - + - + - + - + -
O | O |   |   |   |  

It's Logistic Regression's

- + - + - + - + - + -
X | X | O | O | X | X
- + - + - + - + - + -
O | O | X | X | O | O
- + - + - + - + - + -
X | O | O | O | X | X
- + - + - + - + - + -
O | O | X | X | O | O

It's Logistic Regression's turn!

X | X | O | O |   | X
- + - + - + - + - + -
X | O | X | X | O | O
- + - + - + - + - + -
X | X | O | O | X | X
- + - + - + - + - + -
O | O | X | X | O | O
- + - + - + - + - + -
X | O | O | O | X | X
- + - + - + - + - + -
O | O | X | X | O | O

It's MiniMax's turn!
I have compared  1  combinations and executed the best move out of it. I can't lose, dude!

X | X | O | O | X | X
- + - + - + - + - + -
X | O | X | X | O | O
- + - + - + - + - + -
X | X | O | O | X | X
- + - + - + - + - + -
O | O | X | X | O | O
- + - + - + - + - + -
X | O | O | O | X | X
- + - + - + - + - + -
O | O | X | X | O | O

It's over!
Size of newTrainX and newTrainY:  72 72
Training size:  790
Logictic Regression: Hey! It's learning time. Running towards being a human!
I have learnt lots of new tricks!
Match Dr


  |   |   |   |   |  
- + - + - + - + - + -
  |   |   |   |   |  
- + - + - + - + - + -
  |   |   |   |   |  
- + - + - + - + - + -
  |   |   |   |   |  
- + - + - + - + - + -
  |   |   |   |   |  
- + - + - + - + - + -
O |   |   |   |   |  

It's MiniMax's turn!
I have compared  9330  combinations and executed the best move out of it. I can't lose, dude!

  |   |   |   |   |  
- + - + - + - + - + -
  |   |   |   |   |  
- + - + - + - + - + -
  |   |   |   |   |  
- + - + - + - + - + -
  |   |   |   |   |  
- + - + - + - + - + -
X |   |   |   |   |  
- + - + - + - + - + -
O |   |   |   |   |  

It's Logistic Regression's turn!

  |   |   |   |   |  
- + - + - + - + - + -
  |   |   |   |   |  
- + - + - + - + - + -
  |   |   |   |   |  
- + - + - + - + - + -
O |   |   |   |   |  
- + - + - + - + - + -
X |   |   |   |   |  
- + - + - + - + - + -
O |   |   |   |   |  

It's MiniMax's turn!
I have compared  9303  combinations and executed the best move out of it. I can't lose, dude!

  | 

## Battle 4: Logistic Regression Vs Convolutional Neural Network!

Well, as expected, ConvNet didn't perform well, because it's a very small board to capture necessary patterns. ConvNet performed as same as a random move generator! That doesn't mean that Logistic Regression was very impressive. It won the battle, but the moves were like kid sometimes!

Data has been gereated by placing some random moves against minimax algorithm.

Below is the final battle between Logistic Regression Vs ConvNet!. 

In [25]:
def main():    
    player = randint(1, 2)
    print("Let's begin..")
    
    game = ConnectFourAI(8, ['X', 'O'])
#     game.displayBoard()
    
    miniMaxAlgorithm = MiniMaxAlgorithm(game)
    
    logisticRegressionAlgorithm = LogisticRegressionAlgorithm(game)
    convolutionalNeuralNetworkAlgorithm = ConvolutionalNeuralNetworkAlgorithm(game)
    
    randomPlacementAlgorithm = RandomPlacementAlgorithm(game)
    
    print("======================== Generating data =================================")
    
    generateData (player, game, miniMaxAlgorithm, logisticRegressionAlgorithm, convolutionalNeuralNetworkAlgorithm, randomPlacementAlgorithm)
    
    print("======================== Data has been generated =================================")
    
    logisticRegressionAlgorithm.learningTime()
    convolutionalNeuralNetworkAlgorithm.learningTimeCNN()
    
    print("======================== IT'S BATTLE TIME =================================")
    
    cnnCount = 0
    lrCount = 0
    drawCount = 0
    for i in range (5):
        print ("Battle number: ", i + 1)
        print ("==============================================================")
        print ("ConvNet: ", cnnCount, "Logistic Regression: ", lrCount, "Draw: ", drawCount)
        print ("==============================================================")
        game.clear(8, ['X', 'O'])
        
        while (True):
        
            result = None

            if player == 1:
                print ("It's ConvNet's turn!")
                bestMove = convolutionalNeuralNetworkAlgorithm.findBestCNNMove (player) ###########
                result = game.move(bestMove, player)

            if player == 2:
                print ("It's Logistic Regression's turn!")
                bestMove = logisticRegressionAlgorithm.findBestLogisticMove (player)
                result = game.move(bestMove, player)

            game.generateNewRow()
            game.displayBoard()

            if not result == 2:
                print ("It's over!")
                newTrainX, newTrainY = game.getNewData(game.getWinner())
                
                logisticRegressionAlgorithm.addNewData(newTrainX, newTrainY)
                logisticRegressionAlgorithm.learningTime()
                
                convolutionalNeuralNetworkAlgorithm.addNewDataCNN(newTrainX, newTrainY)
                convolutionalNeuralNetworkAlgorithm.learningTimeCNN()


            if game.getWinner() == 0:
                print("Match Draw!")
                drawCount += 1
                break
            elif game.getWinner() == 1:
                print("ConvNet wins!")
                cnnCount += 1
                break
            elif game.getWinner() == 2:
                print("Logistic Regression wins!")
                lrCount += 1
                break

            player = 2 if (player == 1) else 1
    print ("==============================================================")
    print ("ConvNet: ", cnnCount, "Logistic Regression: ", lrCount, "Draw: ", drawCount)
    print ("==============================================================")
    
def generateData (player, game, miniMaxAlgorithm, logisticRegressionAlgorithm, convolutionalNeuralNetworkAlgorithm, randomPlacementAlgorithm):
    
    # Generating winning and losing data
    for i in range (5):
        game.clear(8, ['X', 'O'])
        
        while (True):
        
            result = None

            if player == 1:
                # print ("It's MiniMax's turn!")
                bestMove = randomPlacementAlgorithm.randomPlacement ()
                result = game.move(bestMove, player)

            if player == 2:
                # print ("It's Random Placement's turn!")
                bestMove = randomPlacementAlgorithm.randomPlacement ()
                result = game.move(bestMove, player)

            game.generateNewRow()
            # game.displayBoard()

            if not result == 2:
                # print ("It's over!")
                newTrainX, newTrainY = game.getNewData(game.getWinner())
                
                logisticRegressionAlgorithm.addNewData(newTrainX, newTrainY)
                convolutionalNeuralNetworkAlgorithm.addNewDataCNN(newTrainX, newTrainY)

            if game.getWinner() == 0:
                # print("Match Draw!")
                break
            elif game.getWinner() == 1:
                # print("Minimax wins!")
                break
            elif game.getWinner() == 2:
                # print("Random Placement wins!")
                break

            player = 2 if (player == 1) else 1
            
    for i in range (5):
        game.clear(8, ['X', 'O'])
        
        while (True):
        
            result = None

            if player == 1:
                # print ("It's MiniMax's turn!")
                bestMove = miniMaxAlgorithm.findBestMiniMaxMove (player)
                result = game.move(bestMove, player)

            if player == 2:
                # print ("It's Random Placement's turn!")
                bestMove = randomPlacementAlgorithm.randomPlacement ()
                result = game.move(bestMove, player)

            game.generateNewRow()
            # game.displayBoard()

            if not result == 2:
                # print ("It's over!")
                newTrainX, newTrainY = game.getNewData(game.getWinner())
                
                logisticRegressionAlgorithm.addNewData(newTrainX, newTrainY)
                convolutionalNeuralNetworkAlgorithm.addNewDataCNN(newTrainX, newTrainY)

            if game.getWinner() == 0:
                # print("Match Draw!")
                break
            elif game.getWinner() == 1:
                # print("Minimax wins!")
                break
            elif game.getWinner() == 2:
                # print("Random Placement wins!")
                break

            player = 2 if (player == 1) else 1
    
    # Generating data for draw matches
    for i in range (5):
        game.clear(8, ['X', 'O'])
        
        while (True):
        
            result = None
            
#             if player == 1:
#                 print ("It's MiniMax1's turn!")
#             if player == 2:
#                 print ("It's MiniMax2's turn!")

            bestMove = miniMaxAlgorithm.findBestMiniMaxMove (player)
            result = game.move(bestMove, player)

            game.generateNewRow()
            # game.displayBoard()

            if not result == 2:
                # print ("It's over!")
                newTrainX, newTrainY = game.getNewData(game.getWinner())
                
                logisticRegressionAlgorithm.addNewData(newTrainX, newTrainY)
                convolutionalNeuralNetworkAlgorithm.addNewDataCNN(newTrainX, newTrainY)

            if game.getWinner() == 0:
                # print("Match Draw!")
                break
            elif game.getWinner() == 1:
                # print("Minimax1 wins!")
                break
            elif game.getWinner() == 2:
                # print("Minimax2 wins!")
                break

            player = 2 if (player == 1) else 1

if __name__=="__main__": 
    main() 

Let's begin..
======================== Generating data =================================
Size of newTrainX and newTrainY:  38 38
Training size:  38
Training size:  38
Size of newTrainX and newTrainY:  44 44
Training size:  82
Training size:  82
Size of newTrainX and newTrainY:  26 26
Training size:  108
Training size:  108
Size of newTrainX and newTrainY:  38 38
Training size:  146
Training size:  146
Size of newTrainX and newTrainY:  58 58
Training size:  204
Training size:  204
I have compared  37448  combinations and executed the best move out of it. I can't lose, dude!
I have compared  37448  combinations and executed the best move out of it. I can't lose, dude!
I have compared  34976  combinations and executed the best move out of it. I can't lose, dude!
I have compared  27512  combinations and executed the best move out of it. I can't lose, dude!
Size of newTrainX and newTrainY:  14 14
Training size:  218
Training size:  218
I have compared  37448  combinations and executed the b

I have compared  37448  combinations and executed the best move out of it. I can't lose, dude!
I have compared  37448  combinations and executed the best move out of it. I can't lose, dude!
I have compared  37448  combinations and executed the best move out of it. I can't lose, dude!
I have compared  37448  combinations and executed the best move out of it. I can't lose, dude!
I have compared  37447  combinations and executed the best move out of it. I can't lose, dude!
I have compared  37411  combinations and executed the best move out of it. I can't lose, dude!
I have compared  36892  combinations and executed the best move out of it. I can't lose, dude!
I have compared  33146  combinations and executed the best move out of it. I can't lose, dude!
I have compared  19607  combinations and executed the best move out of it. I can't lose, dude!
I have compared  19187  combinations and executed the best move out of it. I can't lose, dude!
I have compared  18935  combinations and executed 

I have compared  8614  combinations and executed the best move out of it. I can't lose, dude!
I have compared  7402  combinations and executed the best move out of it. I can't lose, dude!
I have compared  3665  combinations and executed the best move out of it. I can't lose, dude!
I have compared  2885  combinations and executed the best move out of it. I can't lose, dude!
I have compared  3660  combinations and executed the best move out of it. I can't lose, dude!
I have compared  2965  combinations and executed the best move out of it. I can't lose, dude!
I have compared  3899  combinations and executed the best move out of it. I can't lose, dude!
I have compared  3883  combinations and executed the best move out of it. I can't lose, dude!
I have compared  3706  combinations and executed the best move out of it. I can't lose, dude!
I have compared  2957  combinations and executed the best move out of it. I can't lose, dude!
I have compared  1364  combinations and executed the best mo

I have compared  37448  combinations and executed the best move out of it. I can't lose, dude!
I have compared  37448  combinations and executed the best move out of it. I can't lose, dude!
I have compared  37448  combinations and executed the best move out of it. I can't lose, dude!
I have compared  37448  combinations and executed the best move out of it. I can't lose, dude!
I have compared  37447  combinations and executed the best move out of it. I can't lose, dude!
I have compared  37411  combinations and executed the best move out of it. I can't lose, dude!
I have compared  36892  combinations and executed the best move out of it. I can't lose, dude!
I have compared  33146  combinations and executed the best move out of it. I can't lose, dude!
I have compared  19607  combinations and executed the best move out of it. I can't lose, dude!
I have compared  19187  combinations and executed the best move out of it. I can't lose, dude!
I have compared  18935  combinations and executed 

c:\program files\python38\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

c:\program files\python38\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_88 (Conv2D)           (None, 8, 8, 32)          160       
_________________________________________________________________
max_pooling2d_66 (MaxPooling (None, 4, 4, 32)          0         
_________________________________________________________________
conv2d_89 (Conv2D)           (None, 4, 4, 64)          8256      
_________________________________________________________________
max_pooling2d_67 (MaxPooling (None, 2, 2, 64)          0         
_________________________________________________________________
conv2d_90 (Conv2D)           (None, 2, 2, 32)          8224      
_________________________________________________________________
max_pooling2d_68 (MaxPooling (None, 1, 1, 32)          0         
_________________________________________________________________
conv2d_91 (Conv2D)           (None, 1, 1, 16)        


  |   |   |   |   |   |   |  
- + - + - + - + - + - + - + -
  |   |   |   |   |   |   |  
- + - + - + - + - + - + - + -
  |   |   |   |   |   |   |  
- + - + - + - + - + - + - + -
X |   |   |   |   |   |   |  
- + - + - + - + - + - + - + -
X |   |   |   |   |   |   |  
- + - + - + - + - + - + - + -
X |   |   |   |   |   |   |  
- + - + - + - + - + - + - + -
O |   |   |   |   |   |   |  
- + - + - + - + - + - + - + -
X |   | O | O | O |   |   |  

It's Logistic Regression's turn!

  |   |   |   |   |   |   |  
- + - + - + - + - + - + - + -
  |   |   |   |   |   |   |  
- + - + - + - + - + - + - + -
  |   |   |   |   |   |   |  
- + - + - + - + - + - + - + -
X |   |   |   |   |   |   |  
- + - + - + - + - + - + - + -
X |   |   |   |   |   |   |  
- + - + - + - + - + - + - + -
X |   |   |   |   |   |   |  
- + - + - + - + - + - + - + -
O |   | O |   |   |   |   |  
- + - + - + - + - + - + - + -
X |   | O | O | O |   |   |  

It's ConvNet's turn!

  |   |   |   |   |   |   |  
- + - + - +

c:\program files\python38\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Epoch 1/30
34/34 [==============================] - 1s 4ms/step - loss: 0.9772 - accuracy: 0.5819
Epoch 2/30
34/34 [==============================] - 0s 4ms/step - loss: 0.7049 - accuracy: 0.5981
Epoch 3/30
34/34 [==============================] - 0s 4ms/step - loss: 0.4815 - accuracy: 0.7382
Epoch 4/30
34/34 [==============================] - 0s 4ms/step - loss: 0.3778 - accuracy: 0.7536
Epoch 5/30
34/34 [==============================] - 0s 5ms/step - loss: 0.3505 - accuracy: 0.7688
Epoch 6/30
34/34 [==============================] - 0s 4ms/step - loss: 0.3317 - accuracy: 0.8072
Epoch 7/30
34/34 [==============================] - 0s 4ms/step - loss: 0.3220 - accuracy: 0.8385
Epoch 8/30
34/34 [==============================] - 0s 4ms/step - loss: 0.3216 - accuracy: 0.8118
Epoch 9/30
34/34 [==============================] - 0s 4ms/step - loss: 0.3158 - accuracy: 0.8512
Epoch 10/30
34/34 [==============================] - 0s 4ms/step - loss: 0.2729 - accuracy: 0.8785
Epoch 11/30
34/34 [